In [1]:
# We used code from this tutorial:
# https://github.com/susanli2016/Machine-Learning-with-Python/blob/master/Collaborative%20Filtering%20Model%20with%20TensorFlow.ipynb
# And we also used code from this tutorial:
# https://medium.com/@connectwithghosh/recommender-system-on-the-movielens-using-an-autoencoder-using-tensorflow-in-python-f13d3e8d600d
# Then, we integrated these two tutorials and edited the code from each of them in order to create a recommender that allows us to recommend a top 10 list of movies without needing to retrain for each new user

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE

In [3]:
rating = pd.read_csv('/Users/blakemyers/Desktop/Jupyter/ml-1m/ratings.csv', sep = "::", error_bad_lines=False, encoding='latin-1', engine='python')

In [4]:
rating.rename({"1": "userId", "1193": "movieId", "5": "rating", "978300760": "timestamp"}, axis=1, inplace=True)

In [5]:
movie = pd.read_csv("/Users/blakemyers/Desktop/Jupyter/ml-1m/movies.csv", sep = "::", error_bad_lines=False, encoding='latin-1', engine='python')

In [6]:
movie.rename({"1": "movieId", "Toy Story (1995)": "title", "Animation|Children's|Comedy": "genre"}, axis=1, inplace=True)

In [7]:
movie_rating = pd.merge(rating, movie, on = 'movieId')

In [8]:
cols = ['timestamp']

In [9]:
movie_rating.drop(cols, axis=1, inplace=True)

In [10]:
numrate_movie = movie_rating.groupby("title")["rating"].count().reset_index()

In [11]:
numrate_movie.rename({"rating": "ratecount_movie"}, axis=1, inplace=True)

In [12]:
numrate_movie = numrate_movie.query("ratecount_movie >= 20")

In [13]:
ratings20plus = pd.merge(numrate_movie, movie_rating, on = 'title', how = 'inner')

In [14]:
numrate_user = ratings20plus.groupby("userId")["rating"].count().reset_index()

In [15]:
numrate_user.rename({"rating": "ratecount_user"}, axis=1, inplace=True)

In [16]:
numrate_user = numrate_user.query("ratecount_user >= 20")

In [17]:
ur20plus = pd.merge(ratings20plus, numrate_user, on = "userId", how = "inner")

In [18]:
scaler = MinMaxScaler()
ur20plus['rating'] = ur20plus['rating'].values.astype(float) #set rating values as float
rating_scaled = pd.DataFrame(scaler.fit_transform(ur20plus['rating'].values.reshape(-1,1)))
#place the rating values on a scale from -1 to 1
ur20plus['rating'] = rating_scaled

In [19]:
ur20plus = ur20plus.drop_duplicates(['userId', 'title']) #drop duplicates
user_movie_matrix = ur20plus.pivot(index='userId', columns='title', values='rating')
#create matrix (see below)
user_movie_matrix.fillna(0, inplace=True)

In [20]:
X_train, X_test = train_test_split(user_movie_matrix, train_size=0.8)
#split the training data (80%) from the testing data

In [21]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [22]:
num_input = ur20plus['title'].nunique()
# Deciding how many nodes each layer should have
n_nodes_inpl = num_input
n_nodes_hl1  = 256
n_nodes_outl = num_input  
# first hidden layer has num_input*32 weights and 32 biases
hidden_1_layer_vals = {'weights':tf.Variable(tf.random_normal([n_nodes_inpl+1,n_nodes_hl1]))}
# first hidden layer has 784*32 weights and 32 biases
output_layer_vals = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1+1,n_nodes_outl]))}

In [23]:
input_layer = tf.placeholder('float', [None, num_input])
# add a constant node to the first layer
# it needs to have the same shape as the input layer to be able to concatinate it later
input_layer_const = tf.fill( [tf.shape(input_layer)[0], 1] ,1.0  )
input_layer_concat =  tf.concat([input_layer, input_layer_const], 1)
# multiply output of input_layer wth a weight matrix 
layer_1 = tf.nn.sigmoid(tf.matmul(input_layer_concat,\
hidden_1_layer_vals['weights']))
# adding one bias node to the hidden layer
layer1_const = tf.fill( [tf.shape(layer_1)[0], 1] ,1.0  )
layer_concat =  tf.concat([layer_1, layer1_const], 1)
# multiply output of hidden with a weight matrix to get final output
output_layer = tf.matmul( layer_concat,output_layer_vals['weights'])
# output_true shall have the original shape for error calculations
output_true = tf.placeholder('float', [None, num_input])
# define cost function
meansq =    tf.reduce_mean(tf.square(output_layer - output_true))
# define optimizer
learn_rate = 0.1   # how fast the model should learn
optimizer = tf.train.AdagradOptimizer(learn_rate).minimize(meansq)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [24]:
# initialising variables and starting the session
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
# defining batch size, number of epochs and learning rate
batch_size = 100  # how many images to use together for training
hm_epochs =200    # how many times to go through the entire dataset
tot_images = X_train.shape[0] # total number of images

In [25]:
# running the model for a 200 epochs taking 100 users in batches
# total improvement is printed out after each epoch
for epoch in range(hm_epochs):
    epoch_loss = 0    # initializing loss (error) as 0
    
    for i in range(int(tot_images/batch_size)):
        epoch_x = X_train[ i*batch_size : (i+1)*batch_size ]
        _, c = sess.run([optimizer, meansq],\
               feed_dict={input_layer: epoch_x, \
               output_true: epoch_x})
        epoch_loss += c
        
    output_train = sess.run(output_layer,\
               feed_dict={input_layer:X_train})
    output_test = sess.run(output_layer,\
                   feed_dict={input_layer:X_test})
        
    if epoch <= 10 or epoch >= 190:
        print('MSE train', MSE(output_train, X_train),'MSE test', MSE(output_test, X_test))
        print('Epoch', epoch, '/', hm_epochs, 'loss:',epoch_loss)
    elif epoch == 11:
        print('.......................')
    else:
        continue

MSE train 47.33322552707516 MSE test 47.66235407544632
Epoch 0 / 200 loss: 3369.5045585632324
MSE train 31.82029214577246 MSE test 32.12288942770701
Epoch 1 / 200 loss: 1864.8186874389648
MSE train 23.002835928291617 MSE test 23.313781267497642
Epoch 2 / 200 loss: 1308.6255264282227
MSE train 17.31630950714615 MSE test 17.640273979014836
Epoch 3 / 200 loss: 962.9341869354248
MSE train 13.651031132623123 MSE test 13.978598063326821
Epoch 4 / 200 loss: 741.0655899047852
MSE train 11.203037601157735 MSE test 11.54220825776259
Epoch 5 / 200 loss: 595.8883638381958
MSE train 9.489249806754174 MSE test 9.819184408093907
Epoch 6 / 200 loss: 496.85125637054443
MSE train 8.232115272382378 MSE test 8.552192322270109
Epoch 7 / 200 loss: 425.91702032089233
MSE train 7.2709559488057405 MSE test 7.581273095620975
Epoch 8 / 200 loss: 372.77850675582886
MSE train 6.515658207176975 MSE test 6.816627401154747
Epoch 9 / 200 loss: 331.63193130493164
MSE train 5.908172122480018 MSE test 6.195790088397789
E

In [26]:
# a top 10 ranking for a new user (9999991) who rates "Chariots of Fire (1981)" with a 5
ur20plus = ur20plus.append(pd.DataFrame([["Chariots of Fire (1981)",1,9999991,1,5,"genre",1]], columns =ur20plus.columns), ignore_index=True)
scaler = MinMaxScaler()
ur20plus['rating'] = ur20plus['rating'].values.astype(float)
rating_scaled = pd.DataFrame(scaler.fit_transform(ur20plus['rating'].values.reshape(-1,1)))
ur20plus['rating'] = rating_scaled
ur20plus = ur20plus.drop_duplicates(['userId', 'title'])
user_movie_matrix = ur20plus.pivot(index='userId', columns='title', values='rating')
user_movie_matrix.fillna(0, inplace=True)
pred_data = pd.DataFrame()
preds = sess.run(output_layer, feed_dict={input_layer: user_movie_matrix})
pred_data = pred_data.append(pd.DataFrame(preds))
pred_data = pred_data.stack().reset_index(name='rating')
pred_data.columns = ['userId', 'title', 'rating']
users = user_movie_matrix.index.tolist()
movies = user_movie_matrix.columns.tolist()
pred_data['userId'] = pred_data['userId'].map(lambda value: users[value])
pred_data['title'] = pred_data['title'].map(lambda value: movies[value])
keys = ['userId', 'title']
index_1 = pred_data.set_index(keys).index
index_2 = ur20plus.set_index(keys).index
top_ten_ranked = pred_data[~index_1.isin(index_2)]
top_ten_ranked = top_ten_ranked.sort_values(['userId', 'rating'], ascending=[True, False])
top_ten_ranked = top_ten_ranked.groupby('userId').head(10)
top_ten_ranked.loc[top_ten_ranked['userId'] == 9999991]

,userId,title,rating
18298879,9999991,Hear My Song (1991),3.023763
18300570,9999991,West Side Story (1961),2.915480
18298423,9999991,"Doom Generation, The (1995)",2.708492
18300253,9999991,Stonewall (1995),2.351707
18299441,9999991,Modern Times (1936),2.343343
18298516,9999991,Everything You Always Wanted to Know About Sex...,2.338465
18299362,9999991,Matilda (1996),2.268787
18298665,9999991,Freeway (1996),2.265756
18299125,9999991,"Karate Kid, The (1984)",2.200862
18299862,9999991,Puppet Master III: Toulon's Revenge (1991),2.199547


In [27]:
# a top 10 ranking for a new user (9999992) who rates "Hurricane, The (1999)" with a 3
ur20plus = ur20plus.append(pd.DataFrame([["Hurricane, The (1999)",1,9999992,1,3,"genre",1]], columns =ur20plus.columns), ignore_index=True)
scaler = MinMaxScaler()
ur20plus['rating'] = ur20plus['rating'].values.astype(float)
rating_scaled = pd.DataFrame(scaler.fit_transform(ur20plus['rating'].values.reshape(-1,1)))
ur20plus['rating'] = rating_scaled
ur20plus = ur20plus.drop_duplicates(['userId', 'title'])
user_movie_matrix = ur20plus.pivot(index='userId', columns='title', values='rating')
user_movie_matrix.fillna(0, inplace=True)
pred_data = pd.DataFrame()
preds = sess.run(output_layer, feed_dict={input_layer: user_movie_matrix})
pred_data = pred_data.append(pd.DataFrame(preds))
pred_data = pred_data.stack().reset_index(name='rating')
pred_data.columns = ['userId', 'title', 'rating']
users = user_movie_matrix.index.tolist()
movies = user_movie_matrix.columns.tolist()
pred_data['userId'] = pred_data['userId'].map(lambda value: users[value])
pred_data['title'] = pred_data['title'].map(lambda value: movies[value])
keys = ['userId', 'title']
index_1 = pred_data.set_index(keys).index
index_2 = ur20plus.set_index(keys).index
top_ten_ranked = pred_data[~index_1.isin(index_2)]
top_ten_ranked = top_ten_ranked.sort_values(['userId', 'rating'], ascending=[True, False])
top_ten_ranked = top_ten_ranked.groupby('userId').head(10)
top_ten_ranked.loc[top_ten_ranked['userId'] == 9999992]

,userId,title,rating
18301102,9999992,Breakfast of Champions (1999),2.948779
18302323,9999992,"Love Letter, The (1999)",2.667478
18300795,9999992,"American Werewolf in Paris, An (1997)",2.570849
18303691,9999992,Wrongfully Accused (1998),2.473479
18302629,9999992,"Night to Remember, A (1958)",2.405450
18303072,9999992,Saving Private Ryan (1998),2.399654
18300697,9999992,8 1/2 (1963),2.369504
18302054,9999992,Immortal Beloved (1994),2.326427
18302409,9999992,Maximum Overdrive (1986),2.313506
18300865,9999992,At First Sight (1999),2.271835
